In [3]:
import numpy as np
import pandas as pd
import os

os.chdir("../")
pd.set_option("display.max_columns", None)

In [9]:
year = 2022
election_round = "first_round" # european_parliament first_round

In [3]:
# Read full data
if year == 2022:
    df = pd.read_excel(f"data/France/{year}_{election_round}.xlsx")
    df["polling_id"] = df["Code du département"].astype(str) + "-" + df["Code de la commune"].astype(str) + "-" + df["Code du b.vote"].astype(str)
    
elif year == 2017:
    df = pd.read_csv(f"data/France/{year}_{election_round}.txt", delimiter=";", encoding="latin-1")
    df = df.reset_index()
    df.columns = [f"level_{x}" for x in range(len(list(df)))]
    
    df["polling_id"] = df["level_0"].astype(str) + "-" + df["level_4"].astype(str) + "-" +\
        "-" + df["level_22"].astype(str)
    
    if election_round == "runoff":
        candidates = {
            "level_25": "Emmanuel MACRON",
            "level_32": "Marine LE PEN"
        }
        
    else:
        candidates = {
            "level_25": "Nicolas DUPONT-AIGNAN",
            "level_32": "Marine LE PEN",
            "level_39": "Emmanuel MACRON",
            "level_46": "Benoît HAMON",
            "level_53": "Nathalie ARTHAUD",
            "level_60": "Philippe POUTOU",
            "level_67": "Jacques CHEMINADE",
            "level_74": "Jean LASSALLE",
            "level_81": "Jean-Luc MÉLENCHON",
            "level_88": "François ASSELINEAU",
            "level_95": "François FILLON"
        }
        
    df = df.rename(columns=candidates)
    df = df.rename(columns={
        "level_8": "Abstentions",
        "level_12": "Blancs",
        "level_15": "Nuls"})
    df_filtered = df[["polling_id"] + list(candidates.values())]
    
elif year == 2012:
    df = pd.read_csv(f"data/France/{year}.txt", delimiter=";", encoding="latin-1")
    df = pd.DataFrame(np.vstack([df.columns, df]))
    rnd = 1 if election_round == "first_round" else 2
    df = df.reset_index()
    df.columns = [f"level_{x}" for x in range(len(list(df)))]
    df = df[df["level_1"] == rnd].copy()

    df["polling_id"] = df["level_2"].astype(str) + "-" + df["level_3"].astype(str)\
        + "-" + df["level_6"].astype(str) + "-" + df["level_7"].astype(str)

    df["Candidate"] = df["level_13"] + " " + df["level_12"]
    df = df.rename(columns={"level_15": "Votes"})

elif year in [2002, 2007]:
    df = pd.read_csv(f"data/France/{year}.txt", delimiter=";", header=16, encoding="latin-1")
    rnd = 1 if election_round == "first_round" else 2
    df = df.reset_index()
    df.columns = [f"level_{x}" for x in range(len(list(df)))]
    df = df[df["level_0"] == rnd].copy()
    df["level_1"] = df["level_1"].astype(str).str.zfill(2)

    df["polling_id"] = df["level_1"].astype(str) + "-" + df["level_2"].astype(str)\
        + "-" + df["level_4"].astype(str)

    df["Candidate"] = df["level_10"] + " " + df["level_9"]
    df = df.rename(columns={"level_12": "Votes"})

df.head()

C:\Users\cnava\AppData\Local\Temp\ipykernel_10780\1195840770.py:57: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"data/France/{year}.txt", delimiter=";", header=16, encoding="latin-1")


,level_0,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,level_10,level_11,Votes,polling_id,Candidate
0,1,01,1,L'Abergement-Clémenciat,0001,596,528,522,5,BOVÉ,José,BOVE,5,01-1-0001,José BOVÉ
1,1,01,1,L'Abergement-Clémenciat,0001,596,528,522,2,BUFFET,Marie-George,BUFF,3,01-1-0001,Marie-George BUFFET
2,1,01,1,L'Abergement-Clémenciat,0001,596,528,522,3,SCHIVARDI,Gérard,SCHI,1,01-1-0001,Gérard SCHIVARDI
3,1,01,1,L'Abergement-Clémenciat,0001,596,528,522,11,LAGUILLER,Arlette,LAGU,6,01-1-0001,Arlette LAGUILLER
4,1,01,1,L'Abergement-Clémenciat,0001,596,528,522,7,de VILLIERS,Philippe,VILL,20,01-1-0001,Philippe de VILLIERS


In [4]:
df_spoilt = pd.DataFrame()

if year in [2017, 2022]:

    df_spoilt = df[["polling_id", "Abstentions", "Blancs", "Nuls"]]\
        .rename(columns={"Abstentions": "ABSTENTION", "Blancs": "BLANK", "Nuls": "NULL"})\
        .melt(id_vars=["polling_id"], var_name="candidate")

    df_spoilt["flag_candidates"] = 0

df_spoilt.head()

""


In [5]:
if year == 2022:

    if election_round == "runoff":
        df = df.rename(columns={
            "Voix": "Emmanuel MACRON",
            "Unnamed: 32": "Marine LE PEN"
        })

        df_filtered = df[[
            "polling_id",
    #         "Code du département", 
    #         "Libellé de la commune", 
    #         "Code du b.vote", 
            "Emmanuel MACRON",
            "Marine LE PEN", 
        ]]
    else:
        df = df.rename(columns={
            "Voix": "Nathalie ARTHAUD",
            "Unnamed: 32": "Fabien ROUSSEL",
            "Unnamed: 39": "Emmanuel MACRON",
            "Unnamed: 46": "Jean LASSALLE",
            "Unnamed: 53": "Marine LE PEN",
            "Unnamed: 60": "Éric ZEMMOUR",
            "Unnamed: 67": "Jean-Luc MÉLENCHON",
            "Unnamed: 74": "Anne HIDALGO",
            "Unnamed: 81": "Yannick JADOT",
            "Unnamed: 88": "Valérie PÉCRESSE",
            "Unnamed: 95": "Philippe POUTOU",
            "Unnamed: 102": "Nicolas DUPONT-AIGNAN"
        })

        df_filtered = df[[
            "polling_id",
    #         "Code du département", 
    #         "Libellé de la commune", 
    #         "Code du b.vote", 
            "Nathalie ARTHAUD", 
            "Fabien ROUSSEL", 
            "Emmanuel MACRON",
            "Jean LASSALLE", 
            "Marine LE PEN", 
            "Éric ZEMMOUR", 
            "Jean-Luc MÉLENCHON", 
            "Anne HIDALGO",
            "Yannick JADOT",
            "Valérie PÉCRESSE",
            "Philippe POUTOU",
            "Nicolas DUPONT-AIGNAN"
        ]]

In [6]:
if year in [2017, 2022]:
    df_filtered = df_filtered.melt(
        id_vars=["polling_id"], 
        var_name="Candidate", 
        value_name="Votes"
    )

elif year in [2002, 2007, 2012]:
    df_filtered = df[["polling_id", "Candidate", "Votes"]].copy()
# df_filtered["polling_id"] = df_filtered["Libellé de la commune"].astype(str) + "-" + df_filtered["Code du b.vote"].astype(str)

df_filtered.head()

,polling_id,Candidate,Votes
0,01-1-0001,José BOVÉ,5
1,01-1-0001,Marie-George BUFFET,3
2,01-1-0001,Gérard SCHIVARDI,1
3,01-1-0001,Arlette LAGUILLER,6
4,01-1-0001,Philippe de VILLIERS,20


In [7]:
## Check whether there are polling ids with zero votes
zeros = df_filtered.groupby("polling_id").agg({"Votes": "sum"})
zeros = list(zeros[zeros["Votes"] == 0].index)

df_filtered = df_filtered[~df_filtered["polling_id"].isin(zeros)].groupby(["polling_id", "Candidate"]).agg({"Votes": "sum"})        
df_filtered["rate"] = df_filtered.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum())
df_filtered = df_filtered.reset_index()
df_filtered = df_filtered.rename(columns={"Votes": "value", "Candidate": "candidate"})
df_filtered = df_filtered[["polling_id", "candidate", "rate", "value"]]
df_filtered["rank"] = df_filtered.groupby(["polling_id"])["value"].rank("min", ascending=False).astype(int)
df_filtered["flag_candidates"] = 1

In [8]:
data = pd.concat([
    df_filtered,
    df_spoilt
])

data.head()

,polling_id,candidate,rate,value,rank,flag_candidates
0,01-1-0001,Arlette LAGUILLER,0.011494,6,9,1
1,01-1-0001,Dominique VOYNET,0.019157,10,8,1
2,01-1-0001,François BAYROU,0.197318,103,2,1
3,01-1-0001,Frédéric NIHOUS,0.022989,12,7,1
4,01-1-0001,Gérard SCHIVARDI,0.001916,1,12,1


In [9]:
data.to_csv(f"data_output/France/{year}_{election_round}.csv.gz", compression="gzip", index=False)

In [10]:
data.candidate.unique()

array(['Arlette LAGUILLER', 'Dominique VOYNET', 'François BAYROU',
       'Frédéric NIHOUS', 'Gérard SCHIVARDI', 'Jean-Marie LE PEN',
       'José BOVÉ', 'Marie-George BUFFET', 'Nicolas SARKOZY',
       'Olivier BESANCENOT', 'Philippe de VILLIERS', 'Ségolène ROYAL'],
      dtype=object)

In [11]:
# https://ec.europa.eu/eurostat/documents/345175/629341/NUTS2021.xlsx

In [12]:
df_nuts = pd.read_excel(
    "https://ec.europa.eu/eurostat/documents/345175/629341/NUTS2021.xlsx",
    sheet_name="NUTS & SR 2021"
)
df_nuts["country_code"] = df_nuts["Code 2021"].str[0:2]

df_nuts = df_nuts[df_nuts["country_code"] == "FR"]#.fillna(method="ffill")

for col in ["NUTS level 1", "NUTS level 2"]:
    df_nuts[col] = df_nuts[col].fillna(method="ffill")
    
df_nuts.head()

def get_nuts_level(df, level=1):
    output_df = df[["Code 2021", f"NUTS level {level}", "NUTS level"]].dropna()
    output_df = output_df[output_df["NUTS level"] == level]
    output_df = output_df[["Code 2021", f"NUTS level {level}"]]
    output_df = output_df.rename(columns={"Code 2021": f"nuts_{level}", f"NUTS level {level}": f"nuts_{level}_name"})
    output_df = output_df.reset_index(drop=True)

    return output_df

df_nuts_1 = get_nuts_level(df_nuts, level=1)
df_nuts_2 = get_nuts_level(df_nuts, level=2)

df_nuts_3 = df_nuts[["Code 2021", "NUTS level 1", "NUTS level 2", "NUTS level 3"]].dropna()
df_nuts_3 = df_nuts_3.rename(
    columns={
        "Code 2021": "nuts_3", 
        "NUTS level 3": "department",
        "NUTS level 1": "nuts_1_name",
        "NUTS level 2": "nuts_2_name"
    })

df_nuts_3 = pd.merge(df_nuts_3, df_nuts_2, on="nuts_2_name")
df_nuts_3 = pd.merge(df_nuts_3, df_nuts_1, on="nuts_1_name")
df_nuts_3["department"] = df_nuts_3["department"].str.strip().str.replace("’", "'")

df_nuts_3.head()

,nuts_3,nuts_1_name,nuts_2_name,department,nuts_2,nuts_1
0,FR101,Ile-de-France,Ile-de-France,Paris,FR10,FR1
1,FR102,Ile-de-France,Ile-de-France,Seine-et-Marne,FR10,FR1
2,FR103,Ile-de-France,Ile-de-France,Yvelines,FR10,FR1
3,FR104,Ile-de-France,Ile-de-France,Essonne,FR10,FR1
4,FR105,Ile-de-France,Ile-de-France,Hauts-de-Seine,FR10,FR1


In [13]:
df_nuts_3.head()

,nuts_3,nuts_1_name,nuts_2_name,department,nuts_2,nuts_1
0,FR101,Ile-de-France,Ile-de-France,Paris,FR10,FR1
1,FR102,Ile-de-France,Ile-de-France,Seine-et-Marne,FR10,FR1
2,FR103,Ile-de-France,Ile-de-France,Yvelines,FR10,FR1
3,FR104,Ile-de-France,Ile-de-France,Essonne,FR10,FR1
4,FR105,Ile-de-France,Ile-de-France,Hauts-de-Seine,FR10,FR1


In [14]:
df_deps = pd.read_csv("https://raw.githubusercontent.com/alan-eu/french-departments/main/departments.csv")
df_deps = df_deps.rename(columns={"code": "department_id", "name": "department"})
df_deps = df_deps.replace({"Eure-et-Loire": "Eure-et-Loir"})
df_deps["department_id"] = df_deps["department_id"].astype(str).str.zfill(2)
df_deps.head()

,department_id,department,nameWithPreposition
0,01,Ain,dans l’Ain
1,02,Aisne,dans l’Aisne
2,03,Allier,dans l’Allier
3,04,Alpes-de-Haute-Provence,dans les Alpes-de-Haute-Provence
4,05,Hautes-Alpes,dans les Hautes-Alpes


In [22]:
if year in [2007, 2012]:
    df_nuts_3 = pd.merge(df_deps, df_nuts_3, on="department")

In [16]:
set(df_deps.department) - set(df_nuts_3.department)

set()

In [23]:
if year == 2017:

    df_location = df[["polling_id", "level_0", "level_1", "level_2", "level_3", "level_4", "level_5", "level_22"]]

    df_location = df_location.rename(columns={
        "level_0": "department_id",
        "level_1": "department",
        "level_2": "circunscription_id",
        "level_3": "circunscription",
        "level_4": "commune_id",
        "level_5": "commune",
        "level_22": "polling_place"
    })
    
elif year == 2022:
    df_location = df[["Code du département", "Libellé du département", "Code de la circonscription", "Libellé de la circonscription",
       "Code de la commune", "Libellé de la commune", "Code du b.vote", "Inscrits", "Abstentions", "polling_id"]]

    df_location = df_location.rename(columns={
        "Code du département": "department_id",
        "Libellé du département": "department",
        "Code de la circonscription": "circunscription_id",
        "Libellé de la circonscription": "circunscription",
        "Code de la commune": "commune_id",
        "Libellé de la commune": "commune",
        "Code du b.vote": "polling_place",
        "Inscrits": "electors",
        "Abstentions": "abstentions"
    })
    
elif year == 2012:
    df_location = df[["level_2", "level_3", "level_4", "level_8", "polling_id"]].copy().drop_duplicates()

    df_location = df_location.rename(columns={
        "level_2": "department_id",
        "level_3": "commune_id",
        "level_4": "commune",
        "level_8": "electors"
    })
    
elif year in [2002, 2007]:
    df_location = df[["level_1", "level_2", "level_3", "level_4", "level_5", "level_6", "polling_id"]].copy().drop_duplicates()

    df_location = df_location.rename(columns={
        "level_1": "department_id",
        "level_2": "commune_id",
        "level_3": "commune",
        "level_4": "polling_station",
        "level_5": "inscrits",
        "level_6": "voters"
    })

if year in [2017, 2022]:
    df_location = pd.merge(df_location, df_nuts_3, on="department", how="left")

elif year in [2007, 2012]:
    df_location = df_location[["polling_id", "department_id", "commune_id"]].drop_duplicates()
    df_location = pd.merge(df_location, df_nuts_3, on="department_id", how="left").dropna(subset=["department"])
    

df_location["commune_id"] = df_location["department_id"].astype(str).str.zfill(2) + df_location["commune_id"].astype(str).str.zfill(3)
df_location.to_csv(f"data_output/France/{year}_{election_round}_location.csv.gz", compression="gzip", index=False)

In [24]:
df_filtered.shape

(787296, 6)

In [19]:
# dd = pd.merge(df_filtered, df_location[["polling_id", "nuts_2"]].drop_duplicates(), how="inner", on="polling_id")
# dd.groupby("polling_id").count().sort_values("candidate")